In [12]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    roc_curve, auc, precision_recall_curve,
    average_precision_score, classification_report,
    confusion_matrix, ConfusionMatrixDisplay
)
from sklearn.calibration import calibration_curve
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
import time
import cv2

# **CONFIGURATION AND DATA LOADING**

In [13]:
print("--- BLOCK 1: SETUP AND DATA LOADING ---")

# --- Paths and Constants ---
MODEL_PATH = "anemia_model.keras"
TEST_DIR = "dataset/test"
IMG_SIZE = (224, 224)
BATCH_SIZE = 16
AUTOTUNE = tf.data.AUTOTUNE

# --- Load the saved model ---
print(f"Loading model from {MODEL_PATH}...")
model = load_model(MODEL_PATH)
print("✅ Model loaded successfully.")

# --- Create the test dataset (unshuffled for consistent evaluation) ---
def apply_preprocessing(image, label):
    """Applies the MobileNetV3 preprocessing function."""
    return preprocess_input(image), label

test_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR, label_mode='int', image_size=IMG_SIZE,
    batch_size=BATCH_SIZE, shuffle=False
)

class_names = test_ds.class_names
print(f"✅ Class names found: {class_names}")

test_ds = test_ds.map(apply_preprocessing).prefetch(buffer_size=AUTOTUNE)

# --- Get predictions and true labels ---
print("Generating predictions on the test set...")
y_pred_probs = model.predict(test_ds).flatten()
y_true = np.concatenate([y for _, y in test_ds], axis=0)
print("✅ Predictions generated.")

--- BLOCK 1: SETUP AND DATA LOADING ---
Loading model from anemia_model.keras...
✅ Model loaded successfully.
Found 99 files belonging to 2 classes.
✅ Class names found: ['anemia', 'non-anemia']
Generating predictions on the test set...
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 203ms/step
✅ Predictions generated.


# **PERFORMANCE REPORTS**

In [14]:
print("\n--- BLOCK 2: PERFORMANCE ANALYSIS & THRESHOLD OPTIMIZATION ---")

# --- Find which class index corresponds to 'anemia' ---
anemia_class_index = class_names.index('anemia')
print(f"'anemia' corresponds to class index: {anemia_class_index}")

# --- **CRITICAL FIX**: Standardize scores to represent the 'anemia' class probability ---
# This resolves the label inversion bug.
if anemia_class_index == 1:
    y_scores = y_pred_probs # High score means anemia
else: # anemia_class_index == 0
    y_scores = 1 - y_pred_probs # High score (after inversion) means anemia

# Now, 'y_true' labels need to be 1 for 'anemia' and 0 for 'non-anemia'
y_true_positive = (y_true == anemia_class_index).astype(int)

# --- Calculate performance with the default 0.5 threshold ---
print("\n--- Performance with Default Threshold (0.5) ---")
y_pred_default = (y_scores > 0.5).astype(int)
print(classification_report(y_true_positive, y_pred_default, target_names=['non-anemia', 'anemia']))

# --- Automatically find the optimal threshold from the ROC curve ---
fpr, tpr, thresholds = roc_curve(y_true_positive, y_scores)
j_scores = tpr - fpr # Youden's J statistic
best_j_index = np.argmax(j_scores)
OPTIMIZED_THRESHOLD = thresholds[best_j_index]

print(f"\n✅ Automatically found optimal threshold: {OPTIMIZED_THRESHOLD:.4f}")
print("This threshold balances sensitivity and specificity to maximize detection.")

# --- Calculate final performance with the OPTIMIZED threshold ---
print("\n--- FINAL Performance with Optimized Threshold ---")
y_pred_optimized = (y_scores > OPTIMIZED_THRESHOLD).astype(int)
print(classification_report(y_true_positive, y_pred_optimized, target_names=['non-anemia', 'anemia']))


--- BLOCK 2: PERFORMANCE ANALYSIS & THRESHOLD OPTIMIZATION ---
'anemia' corresponds to class index: 0

--- Performance with Default Threshold (0.5) ---
              precision    recall  f1-score   support

  non-anemia       0.79      0.81      0.80        52
      anemia       0.78      0.77      0.77        47

    accuracy                           0.79        99
   macro avg       0.79      0.79      0.79        99
weighted avg       0.79      0.79      0.79        99


✅ Automatically found optimal threshold: 0.1641
This threshold balances sensitivity and specificity to maximize detection.

--- FINAL Performance with Optimized Threshold ---
              precision    recall  f1-score   support

  non-anemia       0.95      0.73      0.83        52
      anemia       0.76      0.96      0.85        47

    accuracy                           0.84        99
   macro avg       0.86      0.84      0.84        99
weighted avg       0.86      0.84      0.84        99



In [15]:
print("\n--- BLOCK 3: GENERATING CONFUSION MATRICES ---")

def plot_confusion_matrices(y_true, y_pred_default, y_pred_optimized, display_labels):
    """Plots side-by-side confusion matrices to show the effect of optimization."""
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Default Threshold Matrix
    cm_default = confusion_matrix(y_true, y_pred_default)
    disp_default = ConfusionMatrixDisplay(confusion_matrix=cm_default, display_labels=display_labels)
    disp_default.plot(ax=axes[0], cmap='Blues')
    axes[0].set_title('Confusion Matrix (Default Threshold = 0.5)')
    
    # Optimized Threshold Matrix
    cm_optimized = confusion_matrix(y_true, y_pred_optimized)
    disp_optimized = ConfusionMatrixDisplay(confusion_matrix=cm_optimized, display_labels=display_labels)
    disp_optimized.plot(ax=axes[1], cmap='Greens')
    axes[1].set_title(f'Confusion Matrix (Optimized Threshold = {OPTIMIZED_THRESHOLD:.2f})')
    
    fig.suptitle('Impact of Threshold Optimization on Predictions', fontsize=16)
    plt.tight_layout()
    plt.show()

# This is the function call that generates "Hình 4.2"
plot_confusion_matrices(y_true_positive, y_pred_default, y_pred_optimized, ['non-anemia', 'anemia'])
print("✅ Confusion matrices generated. Note the decrease in missed anemia cases (bottom-left).")


--- BLOCK 3: GENERATING CONFUSION MATRICES ---


<Figure size 1400x600 with 4 Axes>

✅ Confusion matrices generated. Note the decrease in missed anemia cases (bottom-left).


In [16]:
print("\n--- BLOCK 4: GENERATING CORE PERFORMANCE CURVES ---")

def plot_roc_and_pr_curves(y_true_positive, y_scores):
    plt.figure(figsize=(14, 6))
    # ROC Curve
    fpr, tpr, _ = roc_curve(y_true_positive, y_scores)
    roc_auc = auc(fpr, tpr)
    plt.subplot(1, 2, 1)
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right"); plt.grid(True)
    # Precision-Recall Curve
    precision, recall, _ = precision_recall_curve(y_true_positive, y_scores)
    avg_precision = average_precision_score(y_true_positive, y_scores)
    plt.subplot(1, 2, 2)
    plt.plot(recall, precision, color='blue', lw=2, label=f'PR curve (AP = {avg_precision:.2f})')
    plt.xlabel('Recall'); plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend(loc="lower left"); plt.grid(True)
    plt.tight_layout(); plt.show()

# Use the standardized scores for plotting
plot_roc_and_pr_curves(y_true_positive, y_scores)
print("✅ ROC and PR curves generated.")


--- BLOCK 4: GENERATING CORE PERFORMANCE CURVES ---


<Figure size 1400x600 with 2 Axes>

✅ ROC and PR curves generated.


In [17]:
print("\n--- BLOCK 5: DETAILED PREDICTION SCORE ANALYSIS ---")

def plot_prediction_scores(y_true, y_scores, default_thresh, opt_thresh):
    """
    Creates a strip plot to visualize the distribution of prediction scores
    for each true class, color-coded by prediction correctness.
    """
    # Create masks for each outcome
    tp_mask = (y_true == 1) & (y_scores > opt_thresh)
    tn_mask = (y_true == 0) & (y_scores <= opt_thresh)
    fp_mask = (y_true == 0) & (y_scores > opt_thresh)
    fn_mask = (y_true == 1) & (y_scores <= opt_thresh)

    plt.figure(figsize=(12, 7))
    
    # Plot each point with jitter for better visibility
    plt.scatter(y_scores[tp_mask], np.random.normal(1, 0.08, size=np.sum(tp_mask)), 
                color='green', alpha=0.6, label='True Positive')
    plt.scatter(y_scores[tn_mask], np.random.normal(0, 0.08, size=np.sum(tn_mask)), 
                color='blue', alpha=0.6, label='True Negative')
    plt.scatter(y_scores[fp_mask], np.random.normal(0, 0.08, size=np.sum(fp_mask)), 
                color='orange', alpha=0.8, marker='x', s=80, label='False Positive (Error)')
    plt.scatter(y_scores[fn_mask], np.random.normal(1, 0.08, size=np.sum(fn_mask)), 
                color='red', alpha=0.8, marker='x', s=80, label='False Negative (Error)')

    # Add threshold lines
    plt.axvline(x=default_thresh, color='grey', linestyle='--', label=f'Default Threshold ({default_thresh})')
    plt.axvline(x=opt_thresh, color='purple', linestyle='-', lw=2, label=f'Optimized Threshold ({opt_thresh:.2f})')
    
    plt.yticks([0, 1], ['True Class: Non-Anemia', 'True Class: Anemia'])
    plt.xlabel('Model Prediction Score for "Anemia"')
    plt.title('Detailed Analysis of Individual Prediction Scores')
    plt.legend()
    plt.grid(axis='x', linestyle=':')
    plt.show()

plot_prediction_scores(y_true_positive, y_scores, 0.5, OPTIMIZED_THRESHOLD)
print("✅ Detailed score analysis plot generated.")
print("This plot shows every prediction, revealing where errors occur and the impact of the optimized threshold.")


--- BLOCK 5: DETAILED PREDICTION SCORE ANALYSIS ---


<Figure size 1200x700 with 1 Axes>

✅ Detailed score analysis plot generated.
This plot shows every prediction, revealing where errors occur and the impact of the optimized threshold.


**FUTURE WORK DEMONSTRATION: ROI CROPPING**

In [18]:
print("\n--- BLOCK 4: FUTURE WORK DEMO - ROI CROPPING ---")
print("This demonstrates the critical next step: focusing the model on the correct anatomy.")

def crop_to_roi(image, crop_fraction=0.75):
    """Crops an image to its central region."""
    original_height, original_width, _ = image.shape
    new_height = int(original_height * crop_fraction)
    new_width = int(original_width * crop_fraction)
    
    start_y = (original_height - new_height) // 2
    start_x = (original_width - new_width) // 2
    
    cropped_image = image[start_y:start_y + new_height, start_x:start_x + new_width]
    return cropped_image

# Load one random, un-preprocessed image for the demo
unprocessed_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR, label_mode='int', image_size=IMG_SIZE, batch_size=1, shuffle=True, seed=42
)
sample_img_tensor, _ = next(iter(unprocessed_ds))
sample_img = sample_img_tensor[0].numpy().astype('uint8')

# Apply the crop
cropped_img = crop_to_roi(sample_img)

# Display the comparison
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(sample_img)
plt.title("Original Image (Full View)")
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(cropped_img)
plt.title("ROI Cropped Image (Focused View)")
plt.axis('off')
plt.suptitle("Proposed Pre-processing Step to Improve Model Focus", fontsize=14)
plt.show()
print("✅ ROI cropping demonstration complete.")


--- BLOCK 4: FUTURE WORK DEMO - ROI CROPPING ---
This demonstrates the critical next step: focusing the model on the correct anatomy.
Found 99 files belonging to 2 classes.


<Figure size 1000x500 with 2 Axes>

✅ ROI cropping demonstration complete.


**MODEL EXPLANATION (OCCLUSION SENSITIVITY)**

In [19]:
print("\n--- BLOCK 5: GENERATING MODEL EXPLANATIONS (OCCLUSION MAPS) ---")

def generate_occlusion_sensitivity(model, img, patch_size=20, stride=10):
    img_height, img_width, _ = img.shape
    heatmap = np.zeros((img_height, img_width))
    original_img_preprocessed = preprocess_input(np.expand_dims(img, axis=0))
    baseline_pred = model.predict(original_img_preprocessed, verbose=0)[0][0]
    
    print("Generating Occlusion Map (this may take a minute)...")
    total_patches = len(range(0, img_height - patch_size + 1, stride)) * len(range(0, img_width - patch_size + 1, stride))
    patch_count = 0
    for y in range(0, img_height - patch_size + 1, stride):
        for x in range(0, img_width - patch_size + 1, stride):
            patch_count += 1
            occluded_img = img.copy()
            occluded_img[y:y+patch_size, x:x+patch_size] = 128
            occluded_img_preprocessed = preprocess_input(np.expand_dims(occluded_img, axis=0))
            prediction = model.predict(occluded_img_preprocessed, verbose=0)[0][0]
            score_change = baseline_pred - prediction
            heatmap[y:y+patch_size, x:x+patch_size] += score_change
            if patch_count % 50 == 0:
                print(f"  Processed {patch_count}/{total_patches} patches...", end='\r')
    print(f"\nOcclusion map generated.")
    if np.max(heatmap) > 0:
        heatmap = (heatmap - np.min(heatmap)) / (np.max(heatmap) - np.min(heatmap))
    return heatmap

# Use the shuffled, unprocessed dataset for random examples
for i, (img_tensor, label) in enumerate(unprocessed_ds.take(3)):
    original_img = img_tensor[0].numpy().astype('uint8')
    
    # Get model prediction details using the standardized score
    prediction_prob_raw = model.predict(preprocess_input(np.expand_dims(original_img, axis=0)), verbose=0)[0][0]
    
    if anemia_class_index == 1:
        anemia_score = prediction_prob_raw
    else:
        anemia_score = 1 - prediction_prob_raw
        
    # USE THE OPTIMIZED THRESHOLD FOR THE PREDICTED CLASS LABEL
    prediction_class = 'anemia' if anemia_score > OPTIMIZED_THRESHOLD else 'non-anemia'
    true_class = class_names[label.numpy()[0]]
    
    heatmap = generate_occlusion_sensitivity(model, original_img)
    
    # Display results
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(original_img)
    plt.title(f"Original Image\nTrue Class: {true_class}")
    plt.axis('off')
    plt.subplot(1, 2, 2)
    plt.imshow(original_img, alpha=0.6)
    plt.imshow(heatmap, cmap='jet', alpha=0.4)
    plt.title(f"Occlusion Heatmap\nPredicted: {prediction_class} (Score: {anemia_score:.2f})")
    plt.axis('off')
    plt.suptitle(f"Model Explanation Example {i+1}", fontsize=14)
    plt.tight_layout()
    plt.show()

print("\n✅ All presentation visuals generated successfully.")


--- BLOCK 5: GENERATING MODEL EXPLANATIONS (OCCLUSION MAPS) ---
Generating Occlusion Map (this may take a minute)...
  Processed 400/441 patches...
Occlusion map generated.


<Figure size 1200x600 with 2 Axes>

Generating Occlusion Map (this may take a minute)...
  Processed 400/441 patches...
Occlusion map generated.


<Figure size 1200x600 with 2 Axes>

Generating Occlusion Map (this may take a minute)...
  Processed 400/441 patches...
Occlusion map generated.


<Figure size 1200x600 with 2 Axes>


✅ All presentation visuals generated successfully.
